In [16]:
import numpy as np
from natsort import natsorted
from io import BytesIO
import matplotlib.pyplot as plt
from scipy.optimize import nnls
from IPython.display import clear_output
from numba import jit

plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rc('font', size=12) 
%config InlineBackend.figure_format = 'svg'

# note - if you change settings - beam type, spectrum bin widths, etc, you will need to adjust the plot labels. 
# the value for alphja from the L-curve method, for example, is set by visually looking at that graph. 
# they do not update automatically.

beam_type = 'iso' # use isotropic beam, other options are 'cos' for lambertian or 'beam1d' for pencil beam

# x-ray spectrum discretization. min is 30 keV, max is 1000 keV, min step is 10 keV
x_low_keV =  30   # around 30 keV works well and captures the low energy inflection point;
x_high_keV = 500  # should be left at around 500 keV, X-ray spectra do not usually extend farther
x_bin_keV =  1    # should be left at 1 keV. There is no benefit to coarser binning here and it makes the condition number worse.

# electron spectrum discretization. min is 70 keV, max is 2000 keV, min step is 10 keV
e_low_keV =  50    # suggest 100 keV
e_high_keV = 1000  # should be set to the lowest value you expect from your X-ray data
e_bin_keV =  100    # minimum 10 keV, step size is 10 keV. 

g4_response_data = np.load('/Users/patrick/Documents/phd/data/g4_response.zip')
response = []
file_names = []

# assemble the matrix which maps electron spectra to X-ray spectra by matrix multiplication
for file in natsorted(g4_response_data.files): # arranged by energy
  if (beam_type in file):                      # use the chosen beam type (iso, cos, or beam1d)
    file_names.append(file)
for i in range(int(e_low_keV/10), int((e_high_keV+10)/10), int(e_bin_keV/10)): # energies are sampled in multiples of 10 keV
  response.append(np.loadtxt(BytesIO(g4_response_data[file_names[i]]), dtype='float', delimiter=',', skiprows=9)[x_low_keV:x_high_keV:x_bin_keV,3]/5000000.0)
response = np.array(response).T

# make ranges for X-ray and electron spectrum energies
xspace = np.linspace(x_low_keV,x_high_keV,response.shape[0])
espace = np.linspace(e_low_keV,e_high_keV,response.shape[1])

#  implement tikhonov regularization of different orders with optional non-negativity constraint and preconditioner
def tk(r, alpha, x, order, leftprecon, rightprecon, nneg):
  
  right = []
  for i in range(0, r.shape[1]):
    norm = np.linalg.norm(r[:,i])
    if (norm > 0):
      right.append(1.0/np.linalg.norm(r[:,i]))
    else: 
      right.append(1.0)
  right = np.diag(np.array(right))

  left = []
  for i in range(0, r.shape[0]):
    norm = np.linalg.norm(r[i,:])
    if (norm > 0):
      left.append(1.0/np.linalg.norm(r[i,:]))
    else: 
      left.append(1.0)
  left = np.diag(np.array(left))

  if (not leftprecon):
    left = np.identity(r.shape[0])
  if (not rightprecon):
    right = np.identity(r.shape[1])

  x = np.matmul(left, x)
  R = np.matmul(left,np.matmul(r, right))
  I = np.identity(R.shape[1])          # use for zeroth-order
  L1 = np.copy(I) 
  for i in range(0, R.shape[1] - 1):   # use for first-order
    L1[i,i+1] = -1.0
  L2 = np.copy(I) 
  for i in range(1, R.shape[1]-1):     # use for second-order
    L2[i,i] = -2.0
    L2[i,i+1] = 1.0 
    L2[i,i-1] = 1.0
  L2[0,0] = -2
  L2[0,1] = 1
  L2[-1,-2] = -1
  L2[-1,-1] = -2
  if (order == None):
    op = I*0.0
  elif (order == 0):
    op = I
  elif (order == 1):
    op = L1
  elif (order == 2):
    op = L2
  else:
    return(-1)
  C = np.matmul(np.transpose(R),R) + (alpha**2)*np.matmul(np.transpose(op),op) # normal equations provide explicit solution
  D = np.matmul(np.transpose(R), x)

  if (nneg):
    sol = np.matmul(nnls(C,D,maxiter=10**18)[0],right)      # solve using non-negative least-squares
  else:
    sol = np.matmul(np.linalg.solve(C,D),right)

  error = np.linalg.norm(np.matmul(R,sol) - x)            # difference between the regularized solution and the least-squares solution
  penalty = np.linalg.norm(sol)                           # in zeroth-order tikhonov, the penalty is the norm of the solution
  return(sol,error,penalty)                               # keep track of the error and penalty terms 

# apply leave-one-out cross validation to choose alpha. We choose the alpha that minimizes the error term when x-ray data points are suppressed. 
def cv(r, alphas, x, order, rightprecon, leftprecon, nneg):
    err = []
    for alpha in alphas:
        e = 0
        for i in range(0, len(x)-1,20):                            # this is expensive
            response_truncated = np.delete(response,i,0)       
            x_truncated = np.delete(x,i)                        # delete one X-ray data point and solve the reduced problem
            sol = tk(response_truncated, alpha, x_truncated, order, rightprecon, leftprecon, nneg)[0]
            sol[-1] = 0.0
            e += (np.matmul(response,sol)[i] - x[i])**2 # add the error between the resulting solution and the missing data point
        err.append(e)                                             # this value of alpha gave this total error between fit models and suppressed data points
    return(err) 

In [17]:
def make_random_beam(intensity):
  b = np.full(espace.shape[0], 0.0)

  binrange = range(0,len(espace)-1)
  for j in binrange:
    b[j] = np.random.randint(0,high=10)
  b = b*intensity/np.trapz(b,espace)

  s = np.matmul(response,b)
  for e in range(0, len(xspace)):  # add some measurement noise. This is poisson in reality but use a normal distribution
    s[e] = np.random.normal(s[e], scale=np.sqrt(s[e]))

  return(s,b)

In [26]:
err_1e7_particles_noreg_precon = []
for i in range(0,100):
  beam_signal, beam = make_random_beam(1e7)
  loocv_alphas = np.logspace(-6,0,10)
  loocv = cv(response, loocv_alphas, beam_signal, order=None, leftprecon=True, rightprecon=True, nneg=False)
  loocv_e = np.min(loocv)  
  sol, error, penalty = tk(response, loocv_alphas[np.argmin(loocv_e)], beam_signal, order=None, leftprecon=True, rightprecon=True, nneg=False)
  err_1e7_particles_noreg_precon.append(loocv_e)
  clear_output()
  print('(run 0): solved beam: ' + str(i) + ' (LOOCV error = ' + str(loocv_e) + ')')

err_1e7_particles_order_0_precon = []
for i in range(0,100):
  beam_signal, beam = make_random_beam(1e7)
  loocv_alphas = np.logspace(0,5,10)
  loocv = cv(response, loocv_alphas, beam_signal, order=0, leftprecon=True, rightprecon=True, nneg=False)
  loocv_e = np.min(loocv)  
  sol, error, penalty = tk(response, loocv_alphas[np.argmin(loocv_e)], beam_signal, order=2, leftprecon=True, rightprecon=True, nneg=False)
  err_1e7_particles_order_0_precon.append(loocv_e)
  clear_output()
  print('(run 1): solved beam: ' + str(i) + ' (LOOCV error = ' + str(loocv_e) + ')')

err_1e7_particles_order_1_precon = []
for i in range(0,100):
  beam_signal, beam = make_random_beam(1e7)
  loocv_alphas = np.logspace(0,5,10)
  loocv = cv(response, loocv_alphas, beam_signal, order=1, leftprecon=True, rightprecon=True, nneg=False)
  loocv_e = np.min(loocv)  
  sol, error, penalty = tk(response, loocv_alphas[np.argmin(loocv_e)], beam_signal, order=1, leftprecon=True, rightprecon=True, nneg=False)
  err_1e7_particles_order_1_precon.append(loocv_e)
  clear_output()
  print('(run 2): solved beam: ' + str(i) + ' (LOOCV error = ' + str(loocv_e) + ')')

err_1e7_particles_order_2_precon = []
for i in range(0,100):
  beam_signal, beam = make_random_beam(1e7)
  loocv_alphas = np.logspace(0,5,10)
  loocv = cv(response, loocv_alphas, beam_signal, order=2, leftprecon=True, rightprecon=True, nneg=False)
  loocv_e = np.min(loocv)  
  sol, error, penalty = tk(response, loocv_alphas[np.argmin(loocv_e)], beam_signal, order=2, leftprecon=True, rightprecon=True, nneg=False)
  err_1e7_particles_order_2_precon.append(loocv_e)
  clear_output()
  print('(run 3): solved beam: ' + str(i) + ' (LOOCV error = ' + str(loocv_e) + ')')



(run 3): solved beam: 99 (LOOCV error = 9.008193359391376)


In [44]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=2,hspace=.5,wspace=.3)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])

ax0.hist(err_1e7_particles_noreg_precon,alpha=.5,label='no regularization',log=False,bins=range(1,500,10))
ax1.hist(err_1e7_particles_order_0_precon,alpha=.5,label='zeroth order',log=False,bins=range(1,500,10))
ax2.hist(err_1e7_particles_order_1_precon,alpha=.5,label='first order',log=False,bins=range(1,500,10))
ax3.hist(err_1e7_particles_order_2_precon,alpha=.5,label='second order',log=False,bins=range(1,500,10))

ax0.legend()
ax1.legend()
ax2.legend()
ax3.legend()

ax0.set_title('                                                      LOOCV Residual Histograms (n = 100 random spectra)\n')

ax0.set_xlabel('LOOCV Residuals')
ax0.set_ylabel('count (n = 100)')

ax1.set_xlabel('LOOCV Residuals')
ax1.set_ylabel('count (n = 100)')

ax2.set_xlabel('LOOCV Residuals')
ax2.set_ylabel('count (n = 100)')

ax3.set_xlabel('LOOCV Residuals')
ax3.set_ylabel('count (n = 100)')
plt.savefig('loocv_histograms.pdf')